Task 4 : Bengali using GloVe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

Pre-processing

In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=2e4a62636ea4b85474a8c2fe0ec4d5113d95b48748966ed1401f2e0c5be2eb74
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.6 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.metrics import precision_recall_fscore_support as score
from textblob import TextBlob
import contractions
import emot
import emoji
import demoji
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import nltk
nltk.download('omw-1.4')
pd.set_option("max_colwidth" ,120)
#wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
#stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/bn.txt', encoding = 'utf-8') as f:
    ben_stopwords = f.readlines()
    for i in range(len(ben_stopwords)):
        ben_stopwords[i] = re.sub('\n','', ben_stopwords[i])
stopwords = english_stopwords + ben_stopwords
#english_stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

298

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-13-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punct='“!"#$%&\'।()*+,-./:;<=>?@[\\]^_`{|}~”'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '',text)#remove @ - A Z
    text=contractions.fix(text,slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text= "".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)#this is for not splitting other languages but it is not removing emojis
    if len(text) == 0:
      text= '**********************'
    return text

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/train_BE_AH_HASOC2023.csv')
df2=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/test_BE_AH_HASOC2023.csv')

In [ ]:
df1.head()

,S. No.,text,task_1
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা কেন? এটা কথার কথা? তাহলে কোটা নিয়ে...,NOT
1,2,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদের একটা নিজস্ব কোটা আছে, সেই কোটায় তারা যাকে খুশি চাকরি দিতে পারেন, এ...",NOT
2,3,"বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে বিশ্ববিদ্যালয়,সেমিনার, পাবলিশিং হাউস ও মিডিয়ার। বাংলায় দেখছি গরু চোরাক...",HOF
3,4,প্রতি বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আর আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা। পৃথিবীর কোন সভ্য দে...,NOT
4,5,সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ করে দেয় সমাজকে ছি ছি ছি,HOF


In [ ]:
df2.head()

,S. No.,text
0,1,তাতে তোর কী? বেশি চুলকুনি নারে সালা ?
1,2,অবশ্য চটি চাটা হতে হবে
2,3,"KKR এর প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া কি বাধ্যতামূলক? শাহরুখ KKR এর সহ-মালিক হতে পারে, ক্রিকেটার নয়। ক্রিকেট এবং ক্..."
3,4,হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে। যার যা নাম ধরে ডাকলেই চিনতে পারে।
4,5,এটা নিজের মেয়ের নাম রাখবেন আর এই নাম ধরে ডাকবেন .....😁😅😆


In [ ]:
df1['clean_text'] = df1['text'].apply(lambda x:emo(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:remove_urls(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df2['clean_text'] = df2['text'].apply(lambda x:emo(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:remove_urls(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df1['clean_text'][0:10]

0    শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা এটা কথার কথা তাহলে কোটা নিয়ে প্রাক্তন ...
1    প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ এমপি এদের একটা নিজস্ব কোটা কোটায় তারা যাকে খুশি চাকরি দিতে পারেন এবং চিরকাল এন...
2    বিশ্ব জুড়ে বুদ্ধিজীবিদের সম্পর্ক থাকে বিশ্ববিদ্যালয়সেমিনার পাবলিশিং হাউস মিডিয়ার বাংলায় দেখছি গরু চোরাকারবারিদের স...
3    বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা পৃথিবীর কোন সভ্য দেশের সীমান্...
4                                                                  সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ দেয় সমাজকে ছি ছি ছি
5    যারা প্রেরণার কথার মানে বুঝতে পারেনি তাদের মানে বোঝার জন্য ওর যেতে হবে ঠাণ্ডা মস্তিস্ক নিয়ে ভালো বোঝাতে পারবে আমার ...
6                                                                         কন্ডম দিদি grinning face sweat grinning face sweat
7                                                                                                         বড় বড় কথা কোথায়


In [ ]:
df2['clean_text'][0:10]

0                                                                                              তাতে তোর কী চুলকুনি নারে সালা
1                                                                                                               চটি চাটা হবে
2    kkr প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া বাধ্যতামূলক শাহরুখ kkr সহমালিক পারে ক্রিকেটার ক্রিকেট এবং ক্রিকেটারকে ভালবেসে আপ...
3                                               হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে যার নাম ধরে ডাকলেই চিনতে পারে
4            এটা নিজের মেয়ের নাম রাখবেন নাম ধরে ডাকবেন beaming face smiling eyes grinning face sweat grinning squinting face
5    পিগি চপস মানে বাংলায় ওইটাই বোঝায় face tears joy face tears joy face tears joy face tears joy face tears joy আমার ম...
6                                                                                                  মাছেলে দুজনকেই ভালো লাগছে
7                                                                         ইয়ে মাথায় ওঠেনি এগুলো সবই পরিকল্পনা বিদ্বেষ ছাড়ানো


In [ ]:
# build train and test datasets
txt = df1['clean_text'].values
lab = df1['task_1'].values

train_reviews = txt[:960]
train_sentiments = lab[:960]

test_reviews = txt[960:]
test_sentiments = lab[960:]

In [ ]:
#split the dataset into train and test sets
X_train1, X_test1, y_train1, y_test1= train_test_split(df1.clean_text,df1.task_1,test_size=0.25, random_state=42)

In [ ]:
X_train1.shape, X_test1.shape

((960,), (321,))

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [ ]:
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(train_reviews)
t.word_index['<PAD>'] = 0

In [ ]:
t.word_index

In [ ]:
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']

(('লাগব', 5318), ('<PAD>', 0), 1)

In [ ]:
train_sequences = t.texts_to_sequences(train_reviews)

In [ ]:
train_sequences

In [ ]:
test_sequences = t.texts_to_sequences(test_reviews)

In [ ]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=5319
Number of Documents=960


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
test_lens = [len(s) for s in test_sequences]

fig, ax = plt.subplots(1,2, figsize=(12, 6))
h1 = ax[0].hist(train_lens)
h2 = ax[1].hist(test_lens)

In [ ]:
MAX_SEQUENCE_LENGTH = 1000

In [ ]:
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((960, 1000), (321, 1000))

In [ ]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [ ]:
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)

In [ ]:
y_test

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,

In [ ]:
tt = [1,1,1, 0, 1]
print(le.inverse_transform(tt))

['NOT' 'NOT' 'NOT' 'HOF' 'NOT']


In [ ]:
VOCAB_SIZE = len(t.word_index)

In [ ]:
EMBED_SIZE = 300
EPOCHS=5
BATCH_SIZE=128

In [ ]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         1595700   
                                                                 
 conv1d (Conv1D)             (None, 1000, 128)         153728    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 64)           32832     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 250, 32)           8

In [ ]:
# Fit the model
model.fit(X_train, y_train,
          validation_split=0.1,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=1)

Epoch 1/5
7/7 [==============================] - 18s 2s/step - loss: 0.6851 - accuracy: 0.5648 - val_loss: 0.6642 - val_accuracy: 0.6250
Epoch 2/5
7/7 [==============================] - 17s 2s/step - loss: 0.6799 - accuracy: 0.5949 - val_loss: 0.6617 - val_accuracy: 0.6250
Epoch 3/5
7/7 [==============================] - 16s 2s/step - loss: 0.6648 - accuracy: 0.5949 - val_loss: 0.6586 - val_accuracy: 0.6250
Epoch 4/5
7/7 [==============================] - 16s 2s/step - loss: 0.5732 - accuracy: 0.6042 - val_loss: 0.6396 - val_accuracy: 0.6250
Epoch 5/5
7/7 [==============================] - 16s 2s/step - loss: 0.2943 - accuracy: 0.9213 - val_loss: 0.8760 - val_accuracy: 0.5833


In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

11/11 [==============================] - 2s 134ms/step - loss: 0.9821 - accuracy: 0.5545
Accuracy: 55.45%


In [ ]:
# predictions = model.predict_classes(X_test1).ravel()
predictions = (model.predict(X_test) > 0.5).astype("int32")
predictions[:10]

11/11 [==============================] - 3s 276ms/step


array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

In [ ]:
predictions = ['HOF' if item == 1 else 'NOT' for item in predictions]
predictions[:10]

['HOF', 'NOT', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT']

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['NOT', 'HOF']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

         HOF       0.32      0.33      0.32       129
         NOT       0.54      0.53      0.53       192

    accuracy                           0.45       321
   macro avg       0.43      0.43      0.43       321
weighted avg       0.45      0.45      0.45       321



,NOT,HOF
NOT,42,87
HOF,91,101


**Evaluation for test set**

In [ ]:
X_train2=df1['clean_text']
X_test2=df2['clean_text']
y_train2=df1['task_1']

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [ ]:
t1 = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t1.fit_on_texts(X_train2)
t1.word_index['<PAD>'] = 0

In [ ]:
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t1.word_index['<UNK>']

(('লাগব', 5318), ('<PAD>', 0), 1)

In [ ]:
train_sequences1 = t1.texts_to_sequences(X_train2)

In [ ]:
test_sequences1 = t1.texts_to_sequences(X_test2)

In [ ]:
print("Vocabulary size={}".format(len(t1.word_index)))
print("Number of Documents={}".format(t1.document_count))

Vocabulary size=6476
Number of Documents=1281


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens1 = [len(s) for s in train_sequences1]
test_lens1 = [len(s) for s in test_sequences1]

fig1, ax1 = plt.subplots(1,2, figsize=(12, 6))
h11 = ax[0].hist(train_lens1)
h22 = ax[1].hist(test_lens1)

In [ ]:
MAX_SEQUENCE_LENGTH1 = 1000

In [ ]:
# pad dataset to a maximum review length in words
X_train3 = sequence.pad_sequences(train_sequences1, maxlen=MAX_SEQUENCE_LENGTH)
X_test3 = sequence.pad_sequences(test_sequences1, maxlen=MAX_SEQUENCE_LENGTH)
X_train3.shape, X_test3.shape

((1281, 1000), (320, 1000))

In [ ]:
le1 = LabelEncoder()
num_classes1=2 # positive -> 1, negative -> 0

In [ ]:
y_train3 = le.fit_transform(y_train2)
# y_test3 = le.transform(y_test2)

In [ ]:
VOCAB_SIZE1 = len(t1.word_index)

In [ ]:
EMBED_SIZE1 = 300
EPOCHS1=5
BATCH_SIZE1=128

In [ ]:
# create the model
model1= Sequential()
model1.add(Embedding(VOCAB_SIZE1, EMBED_SIZE1, input_length=MAX_SEQUENCE_LENGTH))
model1.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 300)         1942800   
                                                                 
 conv1d_3 (Conv1D)           (None, 1000, 128)         153728    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 500, 128)         0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 500, 64)           32832     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 250, 32)          

In [ ]:
# Fit the model
model1.fit(X_train3, y_train3,
          # validation_split=0.1,
          epochs=EPOCHS1,
          batch_size=BATCH_SIZE1,
          verbose=1)

Epoch 1/5
11/11 [==============================] - 25s 2s/step - loss: 0.6769 - accuracy: 0.5980
Epoch 2/5
11/11 [==============================] - 23s 2s/step - loss: 0.7143 - accuracy: 0.5980
Epoch 3/5
11/11 [==============================] - 22s 2s/step - loss: 0.6538 - accuracy: 0.5980
Epoch 4/5
11/11 [==============================] - 24s 2s/step - loss: 0.5499 - accuracy: 0.7588
Epoch 5/5
11/11 [==============================] - 23s 2s/step - loss: 0.2553 - accuracy: 0.9407


In [ ]:
# # Final evaluation of the model
# scores1 = model1.evaluate(X_test3, y_test, verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# predictions = model.predict_classes(X_test1).ravel()
predictions1 = (model1.predict(X_test3) > 0.5).astype("int32")
# predictions1[:10]

10/10 [==============================] - 2s 161ms/step


In [ ]:
predictions1 = ['HOF' if item == 1 else 'NOT' for item in predictions1]
predictions1[0:320]

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=predictions1, columns=['predictions1'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run5.csv',index = None)